### <Center> Лабораторна робота №6. <br> Ідентифікація користувача за допомогою логістичної регресії

In [82]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

### 1. Завантаження і перетворення даних
Дані можна самостійно завантажити за посиланням [сторінка](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). Однак можна цього не робити, оскільки дані вже завантажені для проведення лабораторної роботи

In [83]:
# завантажимо навчальну і тестову вибірки
train_df = pd.read_csv('lab6/data/train_sessions.csv',  index_col='session_id')
test_df = pd.read_csv('lab6/data/test_sessions.csv', index_col='session_id')

In [84]:

# приведемо колонку time1, ..., time10 до часового формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# відсортуємо дані за часом
train_df = train_df.sort_values(by='time1')

# подивимося на заголовок навчальної вибірки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В навчальній вибірці містяться наступні ознаки:
    - site1 – індекс першого відвідування сайту в сесії
    - time1 – час відвідування першого сайту в сесії
    - ...
    - site10 – індекс 10-го відвідування сайту в сесії
    - time10 – час відвідування 10-го сайту в сесії
    - target – цільова змінна, 1 для сесій Еліс, 0 для сесій інших користувачів
    
Сесії користувачів виділені таким чимном, щоб вони не можут бути довші півгодини чи 10 сайтів. Тобто сесія вважається закінченою або коли користувач відвідав 10 сайтів підряд або коли сесія зайняла за часом більше 30 хвилин.

В таблиці зустрічаються пропущені значення, це значить, що сесія містить менше, ніж 10 сайтів. Замінимо пропущені значення нулями і приведемо ознаки до цільового типу. Також заванатажимо словник сайтів і подивимося, як він виглядає:

In [85]:
# приведемо колонки site1, ..., site10 до цілочислового формату і замінимо пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# завантажимо словник сайтів
with open(r"lab6/data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словника сайтів
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всього сайтів:', sites_dict_df.shape[0])
sites_dict_df.head()

всього сайтів: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Виділимо цільову змінну і об'єднаємо вибірки, щоб разом привести їх до розрідженого формату.

In [86]:
# наша цільова змінна
y_train = train_df['target']

# об'єднана таблиця вхідних даних
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# індекс, за яким будемо відокремлювати навчальну вибірку від тестової
idx_split = train_df.shape[0]

Для самої першої моделі ми використовуємо лише відвідувані сайти в сесіях (але не будемо звертати увагу на часові ознаки). В основі такого вибору даних для моделей лежить така ідея: * у Еліс є свої улюблені сайти, і якщо ви ще побачите ці сайти в сесіях, тим вище ймовірність, що це сесія Еліс і навпаки. *

Підготуємо дані, з усієї таблиці виберемо лише ознаки `site1, site2, ..., site10`. Нагадуємо, що пропущені значення замінені нулем. Ось як виглядатимуть перші рядки таблиць:

In [87]:
# таблиця з індексами відвіданих сайтів в сесії
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сесії представляють собою послідовність індексів сайтів і дані в такому вигляді невдалі для лінійних методів. Відповідно до нашої гіпотези (у Еліс є улюблені сайти) необхідно перетворити цю таблицю таким чином, щоб кожен можливий веб-сайт відповідав своєму окремому призначенню (колонка), а його значення зростало за кількістю відвідувачів цього веб-сайту в сесіях. Це робиться в два рядки:

In [88]:
from scipy.sparse import csr_matrix

In [89]:
csr_matrix#?

scipy.sparse._csr.csr_matrix

In [90]:
# послідовність з індексами
sites_flatten = full_sites.values.flatten()

# шкана матриця
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

Ще один плюс використання розріджених матриць у тому, що для них є спеціальні реалізації як матричних операцій, так і алгоритми машинного навчання, що дозволяє істотно прискорити операції за рахунок особливостей структур даних. Це стосується і логістичної регресії. Ось тепер у нас все готове для побудови наших перших моделей.

### 2. Побудова першої моделі

Отже, у нас є алгоритм та дані для нього, побудуйте нашу першу модель, використовуючи реалізацію [логістичної регресії] (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) з пакета ` sklearn` з параметрами за замовчуванням. Перші 90% даних будемо використовувати для навчання (навчальна вибірка, відсортована за часом), а також 10% для перевірки якості (validation).

**Напишіть просту функцію, яка поверне якість моделей на вкладеній вибірці, і навчіть наш перший класифікатор**.

In [91]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – вибірка
    ratio – у якому співвідношенні поділити вибірку
    C, seed – коефіцієт регуляризації і random_state 
              логістичної регресії
    '''
    
    # Розділяємо дані на навчальну та валідаційну вибірки
    split_idx = int(X.shape[0] * ratio)
    X_train, X_valid = X[:split_idx, :], X[split_idx:, :]
    y_train, y_valid = y[:split_idx], y[split_idx:]
    
    # Навчаємо модель логістичної регресії
    lr = LogisticRegression(C=C, random_state=seed, solver='liblinear')
    lr.fit(X_train, y_train)
    
    # Робимо прогноз на валідаційній вибірці
    y_pred_proba = lr.predict_proba(X_valid)[:, 1]
    
    return roc_auc_score(y_valid, y_pred_proba)

**Подивіться, який отримано ROC AUC на відкладеній вибірці.**

In [92]:
# Навчаємо перший класифікатор на розріджених даних сайтів
X_train_sparse = full_sites_sparse[:idx_split]

# Отримуємо ROC AUC на відкладеній вибірці
auc_score = get_auc_lr_valid(X_train_sparse, y_train)
print(f"ROC AUC на відкладеній вибірці: {auc_score:.4f}")

ROC AUC на відкладеній вибірці: 0.9195


Будем вважати цю модель нашою першою відправною точкою (базовий рівень). Для побудови моделей для прогнозування на тестовій вибірці ** необхідно навчити модель заново вже на всій навчальній вибірці ** (покищо наша модель навчилася лише на частині даних), що підвищує її узагальнюючу здатність:

In [93]:
# функція для запису прогнозів в файлі
def write_to_submission_file(predicted_labels, out_file, target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels, index = np.arange(1, predicted_labels.shape[0] + 1), columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Навчіть модель на всій вибірці, зробіть прогноз для тестової вибірки і покажіть результат**.

In [ ]:
X_train_full = full_sites_sparse[:idx_split]
X_test_full = full_sites_sparse[idx_split:]

# Створюємо та навчаємо модель логістичної регресії
logit = LogisticRegression(C=1.0, random_state=17, solver='liblinear')
logit.fit(full_sites_sparse[:idx_split], y_train)

# Робимо прогноз на тестовій вибірці
y_test_pred = logit.predict_proba(X_test_full)[:, 1]

write_to_submission_file(y_test_pred, 'lab6/result/submission.csv')

print(y_test_pred[:10])

[2.20990851e-03 4.81035990e-09 1.87260622e-08 2.35490144e-08
 3.13025197e-05 2.18462443e-04 5.47935406e-04 1.32273071e-04
 7.95167494e-04 1.03138077e-01]


### 3. Покращення моделі, побудова нових ознак

Створіть таку ознаку, яка буде представлят собою число формату ГГГГММ від тої дати, коли відбувалась сесія, наприклад 201407 -- 2014 рік і 7 месяц. Таким чином, ми будемо враховувати помісячний [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь період наданих даних.

In [95]:
full_df['start_month'] = full_df['time1'].dt.year * 100 + full_df['time1'].dt.month
print(full_df[['time1', 'start_month']].head(10))

                         time1  start_month
session_id                                 
21669      2013-01-12 08:05:57       201301
54843      2013-01-12 08:37:23       201301
77292      2013-01-12 08:50:13       201301
114021     2013-01-12 08:50:17       201301
146670     2013-01-12 08:50:20       201301
242171     2013-01-12 08:50:22       201301
57157      2013-01-12 08:50:25       201301
240201     2013-01-12 08:50:28       201301
210686     2013-01-12 08:50:31       201301
98804      2013-01-12 08:50:37       201301


Додайте нову ознаку, попередньо нормалізуючи її за допомогою `StandardScaler`, і знову підрахуйте ROC AUC на відкладеній вибірці.

In [96]:
# Нормалізуємо ознаку start_month за допомогою StandardScaler
scaler = StandardScaler()
start_month_scaled = scaler.fit_transform(full_df[['start_month']])

from scipy.sparse import hstack

# Додаємо нормалізовану ознаку start_month до розрідженої матриці сайтів
full_sites_with_month = hstack([full_sites_sparse, start_month_scaled])

# Перетворюємо в csr_matrix для підтримки індексації
full_sites_with_month = csr_matrix(full_sites_with_month)

# Розділяємо дані на навчальну та тестову частини
X_train_with_month = full_sites_with_month[:idx_split]
X_test_with_month = full_sites_with_month[idx_split:]

# Підраховуємо ROC AUC на відкладеній вибірці з новою ознакою
auc_score_with_month = get_auc_lr_valid(X_train_with_month, y_train)

print(f"ROC AUC з додаванням нормалізованої ознаки start_month: {auc_score_with_month:.4f}")

auc_score_sites_only = get_auc_lr_valid(full_sites_sparse[:idx_split], y_train)
print(f"ROC AUC тільки з сайтами: {auc_score_sites_only:.4f}")
print(f"Покращення: {auc_score_with_month - auc_score_sites_only:.4f}")

ROC AUC з додаванням нормалізованої ознаки start_month: 0.9197
ROC AUC тільки з сайтами: 0.9195
Покращення: 0.0002


**Додайте дві нові ознаки: start_hour і morning.**

Ознака `start_hour` - це час у якому почалася сесія (від 0 до 23), а бінарна оознака` morning` рівна 1, якщо сесія почалася вранці і 0, якщо сесія почалася пізніше (будемо вважати, що це ранок, якщо `start_hour рівний` 11 або менше).

**Підрахуйте RUC AUC на відкладеній вибірці для вибірки з:**
- сайтами, `start_month` і` start_hour`
- сайтами, `start_month` і` morning`
- сайтами, `start_month`,` start_hour` і `morning`

In [97]:
# Додаємо ознаку start_hour (час початку сесії від 0 до 23)
full_df['start_hour'] = full_df['time1'].dt.hour

# Додаємо бінарну ознаку morning (1 якщо start_hour <= 11, інакше 0)
full_df['morning'] = (full_df['start_hour'] <= 11).astype(int)

# Нормалізуємо start_hour
scaler_hour = StandardScaler()
start_hour_scaled = scaler_hour.fit_transform(full_df[['start_hour']])

# Нормалізуємо morning (хоча це бінарна ознака, нормалізація не змінить результат)
scaler_morning = StandardScaler()
morning_scaled = scaler_morning.fit_transform(full_df[['morning']])

# 1. Сайти + start_month + start_hour
full_sites_month_hour = hstack([full_sites_sparse, start_month_scaled, start_hour_scaled])
full_sites_month_hour = csr_matrix(full_sites_month_hour)
X_train_month_hour = full_sites_month_hour[:idx_split]

auc_sites_month_hour = get_auc_lr_valid(X_train_month_hour, y_train)
print(f"\nROC AUC (сайти + start_month + start_hour): {auc_sites_month_hour:.4f}")

# 2. Сайти + start_month + morning
full_sites_month_morning = hstack([full_sites_sparse, start_month_scaled, morning_scaled])
full_sites_month_morning = csr_matrix(full_sites_month_morning)
X_train_month_morning = full_sites_month_morning[:idx_split]

auc_sites_month_morning = get_auc_lr_valid(X_train_month_morning, y_train)
print(f"ROC AUC (сайти + start_month + morning): {auc_sites_month_morning:.4f}")

# 3. Сайти + start_month + start_hour + morning
full_sites_month_hour_morning = hstack([full_sites_sparse, start_month_scaled, start_hour_scaled, morning_scaled])
full_sites_month_hour_morning = csr_matrix(full_sites_month_hour_morning)
X_train_month_hour_morning = full_sites_month_hour_morning[:idx_split]

auc_sites_month_hour_morning = get_auc_lr_valid(X_train_month_hour_morning, y_train)
print(f"ROC AUC (сайти + start_month + start_hour + morning): {auc_sites_month_hour_morning:.4f}")


ROC AUC (сайти + start_month + start_hour): 0.9579
ROC AUC (сайти + start_month + morning): 0.9488
ROC AUC (сайти + start_month + start_hour + morning): 0.9592


### 4. Підбір коефіцієнта регуляризації

Отже, ми ввели ознаки, які покращують якість нашої моделі у порівнянні з першим бейслайном. Чи можемо ми домогтися більшого значення метрики? Після того, як ми сформували навчальну та тестову вибірки, майже завжди має сенс підібрати оптимальні гіперпараметри - характеристики моделі, які не змінюються під час навчання. Наприклад, ви вивчали вирішальні дерева, глибина дерева це гіперпараметр, а ознака, за якому відбувається розгалуження і її значення - ні. У використовуваної нами логістичної регресії ваги кожної ознаки змінюються і під час навчання знаходяться їх оптимальні значення, а коефіцієнт регуляризації залишається постійним. Це той гіперпараметр, який ми зараз будемо оптимізувати.

Порахуйте якість на відкладеній вибірці з коефіцієнтом регуляризації, який за замовчуванням `C = 1 ':

In [ ]:
# Використовуємо найкращу комбінацію ознак: сайти + start_month + start_hour + morning
auc_c1 = get_auc_lr_valid(X_train_month_hour_morning, y_train, C=1.0)
print(f"ROC AUC з C=1.0: {auc_c1:.4f}")

ROC AUC з C=1.0: 0.9592


Постараємося побити цей результат за рахунок оптимізації коефіцієнта регуляризації. Візьмемо набір можливих значень C і для кожного з них порахуємо значення метрики на відкладеної вибірці.

Знайдіть `C` з` np.logspace (-3, 1, 10) `, при якому ROC AUC на відкладеної вибірці максимальний.

In [103]:
from sklearn.model_selection import validation_curve

# Набір можливих значень C
C_values = np.logspace(-3, 1, 10)
print("Перевіряємо значення C:", C_values)

auc_scores = []

# Перебираємо всі значення C
for C in C_values:
    auc = get_auc_lr_valid(X_train_month_hour_morning, y_train, C=C)
    auc_scores.append(auc)
    print(f"C = {C:.4f}, ROC AUC = {auc:.4f}")

# Знаходимо найкращий C
best_idx = np.argmax(auc_scores)
best_C = C_values[best_idx]
best_auc = auc_scores[best_idx]

print(f"\nНайкращий коефіцієнт регуляризації: C = {best_C:.4f}")
print(f"Найкращий ROC AUC: {best_auc:.4f}")

Перевіряємо значення C: [1.00000000e-03 2.78255940e-03 7.74263683e-03 2.15443469e-02
 5.99484250e-02 1.66810054e-01 4.64158883e-01 1.29154967e+00
 3.59381366e+00 1.00000000e+01]
C = 0.0010, ROC AUC = 0.8230
C = 0.0028, ROC AUC = 0.8965
C = 0.0077, ROC AUC = 0.9390
C = 0.0215, ROC AUC = 0.9564
C = 0.0599, ROC AUC = 0.9607
C = 0.1668, ROC AUC = 0.9612
C = 0.4642, ROC AUC = 0.9603
C = 1.2915, ROC AUC = 0.9587
C = 3.5938, ROC AUC = 0.9558
C = 10.0000, ROC AUC = 0.9513

Найкращий коефіцієнт регуляризації: C = 0.1668
Найкращий ROC AUC: 0.9612


Нарешті, навчіть модель зі знайденим оптимальним значенням коефіцієнта регуляризації і з побудованими ознаками `start_hour`,` start_month` і `morning`. 

In [ ]:
X_test_final = full_sites_month_hour_morning[idx_split:]

# Навчання моделі з оптимальним C на всій навчальній вибірці
logit_optimized = LogisticRegression(C=best_C, random_state=17, solver='liblinear')
logit_optimized.fit(X_train_month_hour_morning, y_train)

# Прогноз на тестовій вибірці
y_test_pred_optimized = logit_optimized.predict_proba(X_test_final)[:, 1]

write_to_submission_file(y_test_pred_optimized, 'lab6/result/submission_optimized.csv')

print(y_test_pred_optimized[:10])

[3.28566067e-04 2.85339939e-06 9.30993396e-07 6.40061712e-07
 1.98825368e-04 7.07912966e-05 2.94004424e-03 1.85378224e-03
 1.33199241e-04 3.15636324e-03]
